## Requirements and Imports

This notebook does require some external python libraries.

In [ ]:
%pip install pacifica-metadata==0.10.1 pacifica-cli==0.4.0 pacifica-downloader==0.3.5 pacifica-uploader==0.2.5

In [ ]:
import os
import sys
import json
import contextlib
import hashlib
from io import StringIO

import requests

## Load the Test Data Set

The test data set is coupled with the metadata service and
can be loaded through docker like the following.

In [ ]:
from test_data.loadit_test import main

os.environ['METADATA_URL'] = 'http://metadataserver:8121'
main()

## Verify the Services

The verification steps are coupled with the test dataset
loaded from the metadata image.

### The Archive Interface

The archive interface should respond to accessing files.

In [ ]:
test_file = os.path.join('test_data', 'README.md')
for file_id in ['103', '104']:
    with open(test_file) as test_file_fd:
        resp = requests.put('http://archiveinterface:8080/{}'.format(file_id), data=test_file_fd)
        assert resp.status_code == 201

mhash = hashlib.sha1()
with open(test_file) as test_file_fd:
    mhash.update(bytes(test_file_fd.read(), 'utf8'))
hash_sum = mhash.hexdigest()
size = os.stat(test_file).st_size
print(hash_sum)
print(size)

### The Metadata Interface

The metadata service should also be available by docker.

In [ ]:
for file_id in ['103', '104']:
    resp = requests.post('http://metadataserver:8121/files',
        params={'_id': file_id},
        json={"size":size,"hashtype":"sha1","hashsum":hash_sum}
    )
    assert resp.status_code == 200

### The UniqueID Interface

The uniqueid interface needs to be updated with the new files we
inserted prior.

In [ ]:
for mode in ['file', 'upload_job']:
    resp = requests.get('http://uniqueid:8051/getid', params={'mode': mode, 'range': '200'})
    assert resp.status_code == 200
    print(resp.json())

### The Policy Interface

In [ ]:
resp = requests.get('http://policyserver:8181/status/users/search/dmlb2001/simple')
assert resp.status_code == 200
print(json.dumps(resp.json(), sort_keys=True, indent=4))

### The Proxy Interface

In [ ]:
resp = requests.get('http://proxyserver:8180/files/sha1/{}'.format(hash_sum))
assert resp.status_code == 200
new_hash = hashlib.sha1()
new_hash.update(bytes(resp.text, 'utf8'))
assert hash_sum == new_hash.hexdigest()

### Ingest Interface

First we need to configure the CLI.

In [ ]:
os.environ['UPLOAD_URL'] = 'http://ingestfrontend:8066/upload'
os.environ['UPLOAD_STATUS_URL'] = 'http://ingestfrontend:8066/get_state'
os.environ['UPLOAD_POLICY_URL'] = 'http://policyserver:8181/uploader'
os.environ['UPLOAD_VALIDATION_URL'] = 'http://policyserver:8181/ingest'
os.environ['DOWNLOAD_URL'] = 'http://cartfrontend:8081'
os.environ['DOWNLOAD_POLICY_URL'] = 'http://policyserver:8181/status/transactions/by_id'
os.environ['AUTHENTICATION_TYPE'] = 'None'
os.environ['UPLOADER_CONFIG'] = os.path.join('pacifica-cli', 'uploader.json')
from pacifica.cli.__main__ import main
sys.argv = ['cli', 'configure']
main()

Then we can try and upload the test data located in the `test_files` directory.

In [ ]:
from pacifica.cli.__main__ import main
stdout_buf = StringIO()
stderr_buf = StringIO()
with contextlib.redirect_stderr(stderr_buf):
    with contextlib.redirect_stdout(stdout_buf):
        sys.argv = ['cli', 'upload', '--logon=10', 'test_data']
        main()
json_obj_start = stdout_buf.getvalue().find('{')
json_obj_end = stdout_buf.getvalue().find('}')
print(stdout_buf.getvalue()[json_obj_start:json_obj_end+1])
job_id = json.loads(stdout_buf.getvalue()[json_obj_start:json_obj_end+1])['job_id']
print(job_id)

### The Cart Download Interface

In [ ]:
from pacifica.cli.__main__ import main
from tempfile import TemporaryDirectory

with TemporaryDirectory() as tempdir:
    sys.argv = ['cli', 'download', '--transaction-id={}'.format(job_id), '--destination', tempdir]
    main()
    for root, dirs, files in os.walk(tempdir):
        for dirname in dirs:
            print(os.path.relpath(os.path.join(root, dirname), tempdir))
        for filename in files:
            print(os.path.relpath(os.path.join(root, filename), tempdir))